In [1]:
import pandas as pd
df = pd.read_pickle('dataset_with_hadm.pkl')
df.head()

,SUBJECT_ID,HADM_ID,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,22,165315,2,0,65,462.0,0.0,24.0,22.0,1.0,...,0,0,0,0,0,0,0,1,1,0
1,23,152223,6,1,71,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2,23,124321,7,1,75,462.0,1.0,24.0,27.0,1.0,...,0,0,0,0,0,1,0,0,0,1
3,24,161859,3,1,39,462.0,0.0,24.0,27.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,25,129635,4,1,59,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df_nohadm = df.drop(['HADM_ID','SUBJECT_ID'], axis=1)
x = df_nohadm.iloc[:,:483]
y = df_nohadm.iloc[:,483:]
print(x.columns,y.columns)

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=483) Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')


In [19]:
seed=243 # is it gotta do with the prop of the split, shouldnt we regulate that using same seed..
#what do u mean, like each time we are splitting x, y diff.. ya aren't we just storing the trained model
# ya, i mean maybe what was in x_train while in training is now in x_test while predicting, so overfitting
from sklearn.model_selection import train_test_split
train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
#x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size=0.95,random_state=seed)


In [20]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
from sklearn.externals import joblib
rf = joblib.load("./Trained Models/Randomforest-C.pkl")
xgb = joblib.load("./Trained Models/XGBoostC.pkl")
cat = joblib.load("./Trained Models/CatBoostC.pkl")
lgbm = joblib.load("./Trained Models/LGBM-C.pkl")

In [21]:
y_pred_rf = rf.predict(x_test)
y_pred_xgb = xgb.predict(x_test)
y_pred_cat = cat.predict(x_test)
y_pred_lgbm = lgbm.predict(x_test)
xgb_auc= roc_auc_score(y_test,y_pred_xgb)
rf_auc= roc_auc_score(y_test,y_pred_rf)
cat_auc= roc_auc_score(y_test,y_pred_cat)
lgbm_auc= roc_auc_score(y_test,y_pred_lgbm)
print(xgb_auc,lgbm_auc,rf_auc,cat_auc) # wait what! yep!!!
#can u give me 5min sure
# ya tell,wtf lol i mean what did u dowait ill show


0.6673877140694789 0.7449243897449993 0.9514177009908064 0.8434803816891454


In [22]:
xgb_ham = hamming_loss(y_test,y_pred_xgb)
cat_ham = hamming_loss(y_test,y_pred_cat)
lgbm_ham = hamming_loss(y_test,y_pred_lgbm)
rf_ham = hamming_loss(y_test,y_pred_rf)
print(xgb_ham,lgbm_ham,rf_ham,cat_ham)

0.16591499943483667 0.13311291963377417 0.02538148524923703 0.11575675370181983


In [16]:
y_pred_cat.shape

(14744, 20)

In [ ]:
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))

from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold

kf = KFold(train, shuffle=True, random_state=seed)


## Creating Classes for stacking


class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None):   #constructor
        params['random_state'] = seed               #fixes random state
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict_proba(self, x):
        proba = self.clf.predict_proba(x)
        return proba


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 30)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict_proba(self, x):
        proba = self.gbdt.predict(xgb.DMatrix(x))
        return proba


def get_oof(clf):
    oof_train = np.zeros((ntrain,n_classes))
    oof_test = np.zeros((ntest,n_classes))
    oof_test_skf = np.empty((ntest, NFOLDS*n_classes))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)
        oof_train[test_index] = clf.predict_proba(x_te)
        oof_test_skf[:,3*i: 3*i+3] = clf.predict_proba(x_test)
        
    for i in range(3):
        oof_test[:,i] = (oof_test_skf[:,i]+oof_test_skf[:,i+3]+oof_test_skf[:,i+6]+oof_test_skf[:,i+9]+oof_test_skf[:,i+12])/5

    
    return oof_train, oof_test


et_params = {
    'n_jobs': 16,
    'n_estimators': 10,
    'max_features': 0.5,
    'max_depth': 12,
    #'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 10,
    'max_features': 0.2,
    'max_depth': 12,
    #'min_samples_leaf': 2,
}

xgb_params = {
    'objective': 'multi:softprob',
    'eta':0.1,
    'max_depth':6,
    'num_class':3,
    'eval_metric':"mlogloss",
    'min_child_weight': 1,
    'subsample': 0.7,
    'colsample_bytree': 0.7
}



rd_params={
    'alpha': 10
}


ls_params={
    'alpha': 0.005
}


xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)

xg_oof_train, xg_oof_test = get_oof(OneVsRestClassifier(xg))
et_oof_train, et_oof_test = get_oof(OneVsRestClassifier(et))
rf_oof_train, rf_oof_test = get_oof(OneVsRestClassifier(rf))


print("XG-CV: {}".format(log_loss(y_train, xg_oof_train)))
print("ET-CV: {}".format(log_loss(y_train, et_oof_train)))
print("RF-CV: {}".format(log_loss(y_train, rf_oof_train)))


x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test), axis=1)

print("{},{}".format(x_train.shape, x_test.shape))

dtrain = xgb.DMatrix(x_train, label=y_train) #DMatrix? its just like image-generator a form to take in data
dtest = xgb.DMatrix(x_test)

xgb_params = {
    'objective': 'multi:softprob',
    'eta':0.1,
    'max_depth':2,
    'num_class':3,
    'eval_metric':"mlogloss",
    'min_child_weight': 1,
    'subsample': 0.7,
    'colsample_bytree': 0.7
}

res = xgb.cv(xgb_params, dtrain, num_boost_round=50, nfold=5, seed=SEED,
             early_stopping_rounds=10, show_stdv=True)

best_nrounds = res.shape[0] - 1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]


print('Ensemble-CV: {0}+{1}'.format(cv_mean, cv_std))
gbdt = xgb.train(xgb_params, dtrain, best_nrounds)

out_df = pd.DataFrame(gbdt.predict(dtest))
out_df.columns = ["high", "medium", "low"] # eh?
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv('xgstacker_starter.sub.csv', index=False)